In [1]:
import torch
from model import ScaleAdaptiveNet
from data import spectralloader
from torch.optim.lr_scheduler import ReduceLROnPlateau,CosineAnnealingLR
import torch.nn as nn
from utils import Engine,EarlyStop,seed_everything
import numpy as np

# Training Configuration
Example training configureation

In [2]:
CFG = {
    'dataset':'reference', #choose dataset in reference,finetune,test,2018clinical and 2019clnical
    'device':'cuda:0',
    'epoch':1,            #maximun epochs for training,here take 1 epoch as an example
    'batch_size':64,
    'lr':1e-3,
    'wd':1e-5,
    'patience':10,        #patience for early stop
    'sheduler_type':'ReduceLROnPlateau',
    'seed':42,
    'num_classes':30,
    'accumulation_steps':1,#update parameters after forward and backward for 'accumulation_steps'times
    'num_folds':5,
    'mssa_mrsa':False,    #if True, the dataloader will only load the MSSA/MRSA data
    'antibiotics':False,  #if True, the target of the dataset will be the empriric treatment classes
    'fp16':False,         #uing mixed precision for training or not
    'save_path':'param/example/' #saving the params of models in the path
}

## Fixed random seed
Ensure the reproducibility of the experimental results.

In [3]:
seed_everything(CFG['seed'])

## Initialize data-folds

In [4]:
folds = spectralloader(
    dataset=CFG['dataset'],
    batch_size=CFG['batch_size'],
    antibiotics=CFG['antibiotics'],
    num_folds = CFG['num_folds'],
    mssa_mrsa=CFG['mssa_mrsa'],
    seed = CFG['seed']
)

## Training Example
Training for 1 epochs on fold1.
Training and validating by using the tool Class Engine and EarlyStop

In [ ]:
for i in [1]: #replaced with for i in folds:
    trainloader = folds[i]['train']
    valloader = folds[i]['val']
    net = ScaleAdaptiveNet(num_classes=CFG['num_classes']).to(CFG['device'])
    #    net.load_state_dict(torch.load('pretrained_model path')) use this if fine tune

    lossF = nn.BCEWithLogitsLoss().to(CFG['device']) if CFG['mssa_mrsa'] else nn.CrossEntropyLoss().to(CFG['device'])
    optimizer = torch.optim.SGD(net.parameters(),momentum=0.9,lr=CFG['lr'],weight_decay=CFG['wd'])
    sheduler = ReduceLROnPlateau(optimizer,mode='max',factor=0.3,
                                 verbose=False,threshold=0.001,patience=5) if CFG['sheduler_type']=='ReduceLROnPlateau' else\
        CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
        
    es = EarlyStop(patience=CFG['patience'])
    for epoch in range(1,CFG['epoch']+1):
        Engine.train(i,epoch,trainloader,lossF,
                     net,optimizer,CFG['device'],sheduler,CFG['accumulation_steps'],CFG['fp16'],
                    CFG['sheduler_type'],CFG['mssa_mrsa'])
        val_acc,_ = Engine.evaluate(i,epoch,valloader,lossF,net,CFG['device'],CFG['mssa_mrsa'])
        #save the model if val_acc is increasing.
        es(val_acc,net,CFG['save_path']+str(i)+'.pth') 
        #if the val_acc stop increasing for 'patience' epoches, end the training for the fold
        if es.early_stop: 
            break
    

Fold1, Epoch1 : Average train loss is 1.087112, the accuracy rate of trainset is 0.6899 :  74%|███████▍  | 554/750 [00:49<00:17, 11.36it/s]

After training, the model with highest validation accuracy on each fold will be saved in CFG['save_path']

## Config 1
Pretraining on reference dataset for 30 isolate-class task

In [ ]:
CFG = {
    'dataset':'reference', #choose dataset in reference,finetune,test,2018clinical and 2019clnical
    'device':'cuda:0',
    'epoch':100,            #maximun epochs for training
    'batch_size':64,
    'lr':1e-3,
    'wd':1e-5,
    'patience':10,        #patience for early stop
    'sheduler_type':'ReduceLROnPlateau',
    'seed':42,
    'num_classes':30,
    'accumulation_steps':1,#update parameters after forward and backward for 'accumulation_steps'times
    'num_folds':5,
    'mssa_mrsa':False,    #if True, the dataloader will only load the MSSA/MRSA data
    'antibiotics':False,  #if True, the target of the dataset will be the empriric treatment classes
    'fp16':False,         #uing mixed precision for training or not
    'save_path':'param/30/pretrain/' #saving the params of models to the path
}

## Config 2
Finetuning for 30 isolate-class task

In [ ]:
CFG = {
    'dataset':'finetune', #choose dataset in reference,finetune,test,2018clinical and 2019clnical
    'device':'cuda:0',
    'epoch':100,            #maximun epochs for training
    'batch_size':16,
    'lr':1e-4,
    'wd':1e-6,
    'patience':10,        #patience for early stop
    'sheduler_type':'CosineAnnealingLR',
    'seed':42,
    'num_classes':30,
    'accumulation_steps':1,#update parameters after forward and backward for 'accumulation_steps'times
    'num_folds':5,
    'mssa_mrsa':False,    #if True, the dataloader will only load the MSSA/MRSA data
    'antibiotics':False,  #if True, the target of the dataset will be the empriric treatment classes
    'fp16':False,         #uing mixed precision for training or not
    'save_path':'param/30/finetune/' #saving the params of models to the path
}

## Config 3
Pretraining on reference dataset for MSSA/MRSA task

In [ ]:
CFG = {
    'dataset':'reference', #choose dataset in reference,finetune,test,2018clinical and 2019clnical
    'device':'cuda:0',
    'epoch':100,            #maximun epochs for training
    'batch_size':64,
    'lr':1e-3,
    'wd':1e-5,
    'patience':10,        #patience for early stop
    'sheduler_type':'ReduceLROnPlateau',
    'seed':42,
    'num_classes':1,
    'accumulation_steps':1,#update parameters after forward and backward for 'accumulation_steps'times
    'num_folds':5,
    'mssa_mrsa':True,    #if True, the dataloader will only load the MSSA/MRSA data
    'antibiotics':False,  #if True, the target of the dataset will be the empriric treatment classes
    'fp16':False,         #uing mixed precision for training or not
    'save_path':'param/2/pretrain/' #saving the params of models to the path
}

## Config 4
Finetuning for MSSA/MRSA task

In [ ]:
CFG = {
    'dataset':'finetune', #choose dataset in reference,finetune,test,2018clinical and 2019clnical
    'device':'cuda:0',
    'epoch':100,            #maximun epochs for training
    'batch_size':16,
    'lr':1e-4,
    'wd':1e-6,
    'patience':10,        #patience for early stop
    'sheduler_type':'CosineAnnealingLR',
    'seed':42,
    'num_classes':1,
    'accumulation_steps':1,#update parameters after forward and backward for 'accumulation_steps'times
    'num_folds':5,
    'mssa_mrsa':True,    #if True, the dataloader will only load the MSSA/MRSA data
    'antibiotics':False,  #if True, the target of the dataset will be the empriric treatment classes
    'fp16':False,         #uing mixed precision for training or not
    'save_path':'param/2/finetune/' #saving the params of models to the path
}

## Config 5
Pretraining on reference dataset for clinical task

In [ ]:
CFG = {
    'dataset':'reference', #choose dataset in reference,finetune,test,2018clinical and 2019clnical
    'device':'cuda:0',
    'epoch':100,            #maximun epochs for training
    'batch_size':64,
    'lr':1e-3,
    'wd':1e-5,
    'patience':10,        #patience for early stop
    'sheduler_type':'ReduceLROnPlateau',
    'seed':42,
    'num_classes':8,
    'accumulation_steps':1,#update parameters after forward and backward for 'accumulation_steps'times
    'num_folds':5,
    'mssa_mrsa':False,    #if True, the dataloader will only load the MSSA/MRSA data
    'antibiotics':True,  #if True, the target of the dataset will be the empriric treatment classes
    'fp16':False,         #uing mixed precision for training or not
    'save_path':'param/8/pretrain/' #saving the params of models to the path
}